In [6]:
import requests

headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

data = '{ "resource_ids": [ 39435, 39426, 39425, 38657 ], "wikification_type": "SIMPLE"}'

response = requests.post('http://wp3dev.x5gon.org/distance/wikifier/fetch', headers=headers, data=data)


In [7]:
import json
response_c = response.content
dict_response = json.loads(response_c.decode('utf-8'))

In [85]:
import itertools
import math
import trueskill as t
import numpy as np
import json

def return_topic_dificulty(material_id):
    headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
    }

    data ='{ "resource_ids": ['+str(material_id)+'], "wikification_type": "SIMPLE"}'

    response = requests.post('http://wp3dev.x5gon.org/distance/wikifier/fetch', headers=headers, data=data)
    concepts = json.loads(response.content.decode())['output'][0]['value']['concepts']
    concepts_dificulties = {i['title']:i['cosine'] for i in concepts}
    return concepts_dificulties

class TrueBuddy_learner:
    def __init__(self, starting_skill = 0, starting_var = (25/3), BETA = 25/6, inf = 1/1000):
        #starting skill is defenitly 0, starting var has to be determined
        self.st_skill = starting_skill
        self.st_var = starting_var
        self.BETA = BETA
        self.learners = {}
        self.inf = inf
        
    
    
    def update_skills(self, material_id, feedback):
        skills = return_topic_dificulty(material_id)
        for skill in skills.keys():
            if skill in self.learners:
                pass
            else:
                self.learners[skill] = t.Rating(self.st_skill,self.st_var)
        active_list = [self.learners[skill] for skill in skills.keys()]
        
        resource_list = [t.Rating(skills[skill],self.inf) for skill in skills.keys()]
        #print(resource_list)
        
        if feedback == 1:
            active_list, resource_list = t.rate([active_list,resource_list],ranks=[0,1])
        elif feedback == 0:
            active_list, resource_list = t.rate([active_list,resource_list],ranks=[1,0])
        else:
            raise ValueError('feedback can only be 1 or 0')
        
        #print(active_list)
        
        skill_list = list(skills.keys())
        for i in range(len(skill_list)):
            self.learners[skill_list[i]] = active_list[i]
        
        return None

    def win_probability(self,team1, team2):
        delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
        sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
        size = len(team1) + len(team2)
        denom = math.sqrt(size * (self.BETA * self.BETA) + sum_sigma)
        ts = t.global_env()
        return ts.cdf(delta_mu / denom)

    
    
    
    
    
    def predict_proba(self,material_id):
        skills = return_topic_dificulty(material_id)
        for skill in skills.keys():
            if skill in self.learners:
                pass
            else:
                self.learners[skill] = t.Rating(self.st_skill,self.st_var)
        active_list = [self.learners[skill] for skill in skills.keys()]
        
        resource_list = [t.Rating(skills[skill],self.inf) for skill in skills.keys()]
        #print(resource_list)
        
        return(self.win_probability(active_list,resource_list))
        
    
    def predict(self,material_id):
        win_proba = self.predict_proba(material_id)
        
        return(1 if win_proba >= 0.5 else 0)
        
        

In [77]:
l1 = TrueBuddy_learner()

In [84]:
for i in range(1):
    l1.update_skills(39435,1)
l1.predict_proba(39435)

0.9493382280703704

In [1]:
import TrueBuddy as TB

In [2]:
l2 = TB.TrueBuddy_learner()

In [4]:
for i in range(10):
    l2.update_skills(39435,1)
    print(l2.predict_proba(39435))


0.7983983578463263
0.8625094819774723
0.8995165082636599
0.922758850907267
0.9383431718634005
0.9493382280703704
0.9574140664674082
0.9635412550979683
0.968315225908515
0.9721180823692476
